In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 通过Vertex AI Training启动LightGBM

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/get_started_vertex_training_lightgbm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/get_started_vertex_training_lightgbm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/training/get_started_vertex_training_lightgbm.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/>
这份笔记本是[Rajesh Thallam](https://github.com/RajeshThallam/vertex-ai-labs/blob/main/07-vertex-train-deploy-lightgbm/vertex-train-deploy-lightgbm-model.ipynb)的笔记本的修订版。

## 概述

本教程演示了如何使用Vertex AI Training来训练一个LightGBM模型。

了解更多关于[自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

### 目标

在本教程中，您将学习如何使用`Vertex AI Training`来训练一个LightGBM自定义模型。

本教程使用以下Google Cloud ML服务：

- `Vertex AI Training`
- `Vertex AI Model` 资源

执行的步骤包括：

- 使用Python包进行训练。
- 使用GCSFuse将模型工件保存到Cloud Storage。
- 构建一个FastAPI预测服务器。
- 构建一个Dockerfile部署镜像。
- 在本地测试部署镜像。
- 创建一个`Vertex AI Model`资源。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/iris)的[Iris数据集](https://www.tensorflow.org/datasets/catalog/iris)。这个数据集不需要任何特征工程。本教程中的数据集版本存储在公共云存储存储桶中。训练后的模型可以预测三种Iris花种类中的一种：山鸢尾、维吉尼亚或变色鸢尾。

### 成本

本教程使用谷歌云的计费组件：
* Vertex AI
* 云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)生成基于您预计使用量的成本估算。

安装以下软件包以执行此笔记本。

In [ ]:
import os

! pip3 install --upgrade google-cloud-aiplatform  -q
! pip3 install -U google-cloud-storage  -q
! pip3 install -U lightgbm  -q
! pip3 install --upgrade google-auth -q

if os.getenv("IS_TESTING"):
    ! pip3 install --upgrade tensorflow -q

只有在Colab上才能执行：取消注释以下单元格以重启内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

设定您的项目ID

**如果您不知道您的项目ID**，请尝试以下步骤：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### 设置区域

**可选**: 更新 'REGION' 变量以指定您想要使用的区域。了解更多关于[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的谷歌云账户

要验证您的谷歌云账户，请按照您的Jupyter环境的说明进行操作：

**1. Vertex AI Workbench**
<br>您已经通过身份验证。

**2. 本地JupyterLab实例**
<br>取消注释并运行以下代码：

In [ ]:
# ! gcloud auth login

取消注释并运行以下代码：

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

4. 服务账号或其他
* 请参阅如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上为您的服务账号授予Cloud Storage权限。

### 创建一个云存储桶

创建一个存储桶，用于存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时：运行以下单元格即可创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 设置变量

接下来，设置一些在教程中将要使用的变量。
### 导入库和定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
import lightgbm

In [ ]:
print(f"LightGBM version {lightgbm.__version__}")

初始化Python的Vertex SDK

为您的项目和对应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### 启用Artifact Registry API

首先，您必须为您的项目启用Artifact Registry API服务。

了解更多关于[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

### 创建一个私有的Docker仓库

您的第一步是在Google Artifact Registry中创建自己的Docker仓库。

1. 运行`gcloud artifacts repositories create`命令，使用您的地区创建一个新的Docker仓库，并添加描述“docker仓库”。

2. 运行`gcloud artifacts repositories list`命令来验证您的仓库是否已经创建。

In [ ]:
PRIVATE_REPO = "prediction"

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Prediction repository"

! gcloud artifacts repositories list

### 配置私人仓库的身份验证

在推送或拉取容器镜像之前，配置Docker使用`gcloud`命令行工具，对您区域的`Artifact Registry`进行身份验证。

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

设置预先构建的容器

为训练和预测设置预先构建的Docker容器镜像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[预测用的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TRAIN_VERSION = "scikit-learn-cpu.0-23"
DEPLOY_VERSION = "lightgbm-cpu"

# prebuilt
TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)

DEPLOY_IMAGE = "{}-docker.pkg.dev/{}/{}/{}:latest".format(
    REGION, PROJECT_ID, PRIVATE_REPO, DEPLOY_VERSION
)
print("Deploy image:", DEPLOY_IMAGE)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 将变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`设置为配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\]的数量

*注意：以下类型不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

### 检查培训包

#### 包布局

在开始培训之前，您将查看Python包如何为自定义培训工作流程进行组装。解压缩后，包含以下目录/文件布局的包。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的指令。

文件`trainer/task.py`是用于执行自定义培训工作流程的Python脚本。*注意*，当在工作线程池规格中引用它时，我们将目录斜杠替换为点号(`trainer.task`)，并丢弃文件后缀(`.py`)。

#### 包组装

在以下单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n'lightgbm'    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Iris tabular classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 为Python培训包创建任务脚本

接下来，您需要创建`task.py`脚本来驱动培训包。一些值得注意的步骤包括:

- 命令行参数:
    - `model-dir`: 保存训练模型的位置。当使用Vertex AI自定义培训时，位置将在环境变量`AIP_MODEL_DIR`中指定。
    
- 数据预处理(`get_data()`):
    - 下载数据集并拆分为训练集和测试集。
- 训练(`train_model()`):
    - 训练模型
- 模型艺术品保存
    - 将模型艺术品和评估指标保存在由`model-dir`指定的云存储位置。

In [ ]:
%%writefile custom/trainer/task.py
# Single Instance Training for Iris

import datetime
import os
import subprocess
import sys

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import pandas as pd

import lightgbm as lgb

import argparse
import logging

logging.getLogger().setLevel(logging.INFO)

logging.info("Parsing arguments")

parser = argparse.ArgumentParser()
parser.add_argument(
    '--model-dir', 
    dest='model_dir',        
    default=os.getenv('AIP_MODEL_DIR'), 
    type=str, 
    help='Location to export GCS model')
args = parser.parse_args()
logging.info(args)

def get_data():
    # Download data
    logging.info("Downloading data")
    iris = load_iris()
    print(iris.data.shape)

    # split data
    print("Splitting data into test and train")
    x, y = iris.data, iris.target
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

    # create dataset for lightgbm
    print("creating dataset for LightGBM")
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)
    
    return lgb_train, lgb_eval

def train_model(lgb_train, lg_eval):
    # specify your configurations as a dict
    params = {
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': {'multi_error'},
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0,
        'num_class' : 3
    }

    # train lightgbm model
    logging.info('Starting training...')
    model = lgb.train(params,
                    lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval)
    
    return model

lgb_train, lgb_eval = get_data()
model = train_model(lgb_train, lgb_eval)

# GCSFuse conversion
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)
        
# save model to file
logging.info('Saving model...')
model_filename = 'model.txt'
gcs_model_path = os.path.join(args.model_dir, model_filename)
model.save_model(gcs_model_path)

将培训脚本存储在您的云存储桶中

接下来，您可以将培训文件夹打包成一个压缩的tar文件，然后将它存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_iris.tar.gz

### 创建和运行自定义训练作业

要训练一个自定义模型，您需要执行两个步骤：1) 创建一个自定义训练作业，2) 运行作业。

#### 创建自定义训练作业

使用`CustomTrainingJob`类创建一个自定义训练作业，包括以下参数：

- `display_name`：自定义训练作业的人类可读名称。
- `container_uri`：训练容器镜像。

- `python_package_gcs_uri`：Python训练包的位置，以tarball格式存储。
- `python_module_name`：Python包中训练脚本的相对路径。

*注意*：没有requirements参数。您可以在Python包的`setup.py`脚本中指定任何要求。

In [ ]:
DISPLAY_NAME = "iris_"

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_iris.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    project=PROJECT_ID,
)

### 准备你的命令行参数

现在为你的自定义训练容器定义命令行参数:

- `args`: 传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir` : 对于我们的演示，我们使用这个命令行参数来指定存储模型工件的位置。
    - 直接: 将 Cloud Storage 位置作为命令行参数传递给你的训练脚本（设置变量 `DIRECT = True`），或
    - 间接: 服务将 Cloud Storage 位置作为环境变量 `AIP_MODEL_DIR` 传递给你的训练脚本（设置变量 `DIRECT = False`）。在这种情况下，你告诉服务作业规范中的模型工件位置。

In [ ]:
MODEL_DIR = "{}/model".format(BUCKET_URI)

DIRECT = False
if DIRECT:
    CMDARGS = [
        "--model_dir=" + MODEL_DIR,
    ]
else:
    CMDARGS = []

运行自定义训练任务

接下来，通过调用方法`run`，使用以下参数运行自定义作业来开始训练作业：

- `args`：传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作副本的加速器数量。
- `base_output_dir`：用于写入模型工件的Cloud Storage位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
job.run(
    args=CMDARGS,
    replica_count=1,
    machine_type=TRAIN_COMPUTE,
    base_output_dir=MODEL_DIR,
    sync=False,
)

model_path_to_deploy = MODEL_DIR + "/model"

### 列出一个定制的培训工作

In [ ]:
_job = job.list(filter="display_name=iris")
print(_job)

### 等待自定义训练作业完成

接下来，等待自定义训练作业完成。或者，可以在`run()`方法中将参数`sync`设置为`True`，以阻塞直到自定义训练作业完成。

In [ ]:
job.wait()

删除一个自定义训练作业

在训练作业完成后，您可以使用`delete()`方法删除训练作业。在完成之前，可以使用`cancel()`方法取消训练作业。

In [ ]:
job.delete()

### 验证模型工件

接下来，请验证训练脚本是否成功将训练好的模型保存到您的云存储位置。

In [ ]:
print(f"Model path with trained model artifacts {model_path_to_deploy}")

! gsutil ls $model_path_to_deploy

将LightGBM模型部署到Vertex AI端点

使用FastAPI构建一个HTTP服务器

接下来，您可以使用FastAPI将HTTP服务器实现为自定义部署容器。该容器必须监听并响应存活检查，健康检查和预测请求。HTTP服务器必须在0.0.0.0上监听请求。

了解更多关于部署容器要求的信息。

了解更多关于FastAPI。

In [ ]:
# Make folder for serving script
! rm -rf serve
! mkdir serve

# Make the predictor subfolder
! mkdir serve/app

创建用于服务容器的要求文件

接下来，为服务器环境创建`requirements.txt`文件，指定需要安装在服务容器上的Python软件包。

In [ ]:
%%writefile serve/requirements.txt

numpy
scikit-learn>=0.24
pandas
lightgbm==3.2.1
google-cloud-storage


### 编写 FastAPI 服务脚本

接下来，您可以使用 `FastAPI` 编写 HTTP 服务器的服务脚本，如下所示：

- `app`: 实例化一个 `FastAPI` 应用程序
- `health()`: 定义对健康请求的响应。
    - 返回状态码 200
- `predict()`: 定义对预测请求的响应。
    - `body = await request.json()`: 异步等待 HTTP 请求。
    - `instances = body["instances"]` : 预测请求的内容。
    - `inputs = np.asarray(instances)`: 将预测请求重新格式化为 numpy 数组。
    - `outputs = model.predict(inputs)`: 调用模型进行预测。
    - `return {"predictions": ... }`: 在响应主体中返回格式化的预测结果。

In [ ]:
%%writefile serve/app/main.py
from fastapi.logger import logger
from fastapi import FastAPI, Request

import numpy as np
import os

from sklearn.datasets import load_iris
import lightgbm as lgb

import logging

gunicorn_logger = logging.getLogger('gunicorn.error')
logger.handlers = gunicorn_logger.handlers

if __name__ != "main":
    logger.setLevel(gunicorn_logger.level)
else:
    logger.setLevel(logging.DEBUG)

app = FastAPI()

model_f = "/model/model.txt"

logger.info("Loading model")
_model = lgb.Booster(model_file=model_f)
logger.info("Loading target class labels")
_class_names = load_iris().target_names

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    """ health check to ensure HTTP server is ready to handle 
        prediction requests
    """
    return {"status": "healthy"}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()

    instances = body["instances"]
    inputs = np.asarray(instances)
    
    outputs = _model.predict(inputs)

    logger.info(f"Outputs {outputs}")
    return {"predictions": [_class_names[class_num] for class_num in np.argmax(outputs, axis=1)]}

### 添加预启动脚本
FastAPI将在启动服务器之前执行此脚本。 为了在AI平台（Unified）上以期望的端口运行FastAPI，将`PORT`环境变量设置为等于`AIP_HTTP_PORT`。

In [ ]:
%%writefile serve/app/prestart.sh

#!/bin/bash
export PORT=$AIP_HTTP_PORT

### 存储测试实例

接下来，您将创建合成示例，随后测试 FastAPI 服务器和训练后的 LightGBM 模型。

了解有关[自定义模型预测请求的 JSON 格式](https://cloud.google.com/ai-platform-unified/docs/predictions/online-predictions-custom-models#request-body-details)。

In [ ]:
%%writefile serve/instances.json
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
}

## 构建并推送预测容器到Artifact Registry

编写Dockerfile，使用`tiangolo/uvicorn-gunicorn-fastapi`作为基础镜像。这将自动运行 FastAPI，使用 Gunicorn 和 Uvicorn。

了解更多关于[使用Docker部署FastAPI](https://fastapi.tiangolo.com/deployment/docker/)。

In [ ]:
%%bash -s $MODEL_DIR

MODEL_DIR=$1

mkdir -p ./serve/model/
gsutil cp -r ${MODEL_DIR}/model/ ./serve/model/ 

cat > ./serve/Dockerfile <<EOF
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7

COPY ./app /app
COPY ./model /
COPY requirements.txt requirements.txt

RUN pip3 install -r requirements.txt

EXPOSE 7080
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "7080"]

EOF

构建容器本地

接下来，您需要构建并打标签自定义部署容器。

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if not IS_COLAB:
    ! docker build --tag={DEPLOY_IMAGE} ./serve
else:
    # install docker daemon
    ! apt-get -qq install docker.io

在本地以分离模式运行和测试容器（可选）

在本地以分离模式运行容器，并提供容器所需的环境变量。这些环境变量将在部署后由 AI 平台预测提供给容器。测试`/health`和`/predict`路由，然后停止运行的镜像。

在将容器映像推送到工件存储库以与顶点预测一起使用之前，您可以在本地环境中将其作为容器运行，以验证服务器是否按预期工作。

In [ ]:
if not IS_COLAB:
    ! docker rm local-iris 2>/dev/null
    ! docker run -t -d --rm -p 7080:7080 \
        --name=local-iris \
        -e AIP_HTTP_PORT=7080 \
        -e AIP_HEALTH_ROUTE=/health \
        -e AIP_PREDICT_ROUTE=/predict \
        -e AIP_STORAGE_URI={MODEL_DIR} \
        {DEPLOY_IMAGE}
    ! docker container ls
    ! sleep 10

健康检查

向容器的服务器发送健康检查。输出应为{"status": "healthy"}。

In [ ]:
if not IS_COLAB:
    ! curl http://localhost:7080/health

如果成功，服务器将返回以下响应：

```
{
  "status": "healthy"
}
```

检查预测

向容器的服务器发送一个预测请求。

In [ ]:
! curl -X POST \
  -d @serve/instances.json \
  -H "Content-Type: application/json; charset=utf-8" \
  http://localhost:7080/predict

这个请求使用了一个测试句子。如果成功，服务器会以下面的格式返回预测结果：

```
{"predictions":["versicolor","setosa"]}
```

请停止本地容器。

In [ ]:
if not IS_COLAB:
    ! docker stop local-iris

将服务容器推送到Artifact Registry

将带有推理代码和依赖项的容器映像推送到Artifact Registry。

In [ ]:
if not IS_COLAB:
    ! docker push $DEPLOY_IMAGE

在Colab中执行

In [ ]:
%%bash -s $IS_COLAB $DEPLOY_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker build --tag={DEPLOY_IMAGE} ./serve
docker push $2
kill $(jobs -p)

将服务容器部署到 Vertex 预测
我们在 Vertex AI 上创建一个模型资源，并将模型部署到 Vertex Endpoints。在使用模型之前，您必须将模型部署到一个端点。部署的模型会运行自定义容器映像来提供预测。

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到`Model`资源，参数如下：

- `display_name`：`Model`资源的人类可读名称。
- `artifact`：训练模型工件的云存储位置。
- `serving_container_image_uri`：服务容器的镜像。
- `serving_container_predict_route`：发送预测请求到容器的HTTP路径。
- `serving_container_health_route`：发送健康检查请求到容器的HTTP路径。
- `serving_container_ports`：容器监听请求的端口。
- `sync`：是否异步或同步执行上传。

如果`upload()`方法以异步方式运行，您可以随后使用`wait()`方法阻塞直到完成。

In [ ]:
APP_NAME = "iris"

model_display_name = f"{APP_NAME}"
model_description = "LightGBM based iris flower classifier with custom container"

MODEL_NAME = APP_NAME
health_route = "/ping"
predict_route = "/predict"
serving_container_ports = [7080]

In [ ]:
if not os.getenv("IS_TESTING"):
    model = aiplatform.Model.upload(
        display_name=model_display_name,
        description=model_description,
        serving_container_image_uri=DEPLOY_IMAGE,
        serving_container_predict_route=predict_route,
        serving_container_health_route=health_route,
        serving_container_ports=serving_container_ports,
    )

    model.wait()

    print(model.display_name)
    print(model.resource_name)

进行批量预测

文档 - [批量预测请求 - Vertex AI](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

制作测试项

你将使用合成数据作为测试数据项。不要担心我们使用合成数据--我们只是想展示如何进行预测。

In [ ]:
INSTANCES = [[6.7, 3.1, 4.7, 1.5], [4.6, 3.1, 1.5, 0.2]]

制作批量输入文件

现在制作一个批量输入文件，您将把它存储在本地云存储桶中。预测请求中的每个实例都是以下形式的列表：

                        [ [ 内容_1], [内容_2] ]

- `内容`: 测试项的特征值列表。

In [ ]:
import json

[json.dumps(record) for record in INSTANCES]

In [ ]:
import tensorflow as tf

gcs_input_uri = f"{BUCKET_URI}/{APP_NAME}/test/batch_input/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for i in INSTANCES:
        f.write(str(i) + "\n")

! gsutil cat $gcs_input_uri

### 发起批量预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict()方法发起批量预测，使用以下参数：

- `job_display_name`: 批量预测作业的可读名称。
- `gcs_source`: 一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`: 用于存储批量预测结果的Cloud Storage位置。
- `instances_format`: 输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`: 输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`: 用于训练的机器类型。
- `sync`: 如果设置为True，则调用将会阻塞，直到异步批处理作业完成。

In [ ]:
if not os.getenv("IS_TESTING"):
    MIN_NODES = 1
    MAX_NODES = 1

    batch_predict_job = model.batch_predict(
        job_display_name=f"{APP_NAME}",
        gcs_source=gcs_input_uri,
        gcs_destination_prefix=f"{BUCKET_URI}/{APP_NAME}/test/batch_output/",
        instances_format="jsonl",
        predictions_format="jsonl",
        model_parameters=None,
        machine_type=DEPLOY_COMPUTE,
        starting_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
        sync=False,
    )

    print(batch_predict_job)

### 等待批量预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
if not os.getenv("IS_TESTING"):
    batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的Cloud Storage输出桶中。您可以调用方法`iter_outputs()`来获取包含结果的每个云存储文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
import json

if not os.getenv("IS_TESTING"):
    bp_iter_outputs = batch_predict_job.iter_outputs()

    prediction_results = list()
    for blob in bp_iter_outputs:
        if blob.name.split("/")[-1].startswith("prediction"):
            prediction_results.append(blob.name)

    tags = list()
    for prediction_result in prediction_results:
        gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
        with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
            for line in gfile.readlines():
                line = json.loads(line)
                print(line)
                break

进行在线预测

文档 - [在线预测请求](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

部署模型

接下来，部署您的模型以进行在线预测。要部署模型，您需要调用`deploy`方法，并提供以下参数：

- `deployed_model_display_name`：部署模型的可读名称。
- `traffic_split`：在端点上流入此模型的流量百分比，以字典形式指定一个或多个键值对。
如果只有一个模型，则指定为{ "0": 100 }，其中"0"指该模型已上传，100表示100%的流量。
如果端点上有现有模型，要对流量进行分割，则使用model_id指定为{ "0": 百分比, model_id: 百分比, ... }，其中model_id是已部署端点的现有模型的模型id。百分比必须累加等于100。
- `machine_type`：用于训练的机器类型。
- `starting_replica_count`：初始配备的计算实例数量。
- `max_replica_count`：要扩展到的计算实例的最大数量。在本教程中，仅配备一个实例。

In [ ]:
DEPLOYED_NAME = f"{APP_NAME}"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if not os.getenv("IS_TESTING"):
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

### 进行预测

现在您的`Model`资源已部署到`Endpoint`资源，您可以通过向`Endpoint`资源发送预测请求进行在线预测。

#### 请求

每个实例的格式为：

    [feature_list]

由于predict()方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

从predict()调用的响应是一个包含以下条目的Python字典：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个类别标签的预测置信度，介于0和1之间。
- `deployed_model_id`：进行预测的已部署`Model`资源的Vertex AI标识符。

In [ ]:
instances_list = INSTANCES

if not os.getenv("IS_TESTING"):
    prediction = endpoint.predict(instances_list)
    print(prediction)

解除部署模型

当你完成预测后，你可以从`Endpoint`资源中解除部署模型。这将释放所有计算资源并停止对部署模型的计费。

In [ ]:
if not os.getenv("IS_TESTING"):
    endpoint.undeploy_all()

删除您的私有Docker存储库

最后，一旦您的私有存储库变得过时，请使用命令 `gcloud artifacts repositories delete` 在 `Google Artifact Registry` 中删除它。

In [ ]:
! gcloud artifacts repositories delete {PRIVATE_REPO} --location={REGION} --quiet

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Delete the model using the Vertex model object
try:
    model.delete()
    endpoint.delete()
except Exception as e:
    print(e)

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI